In [1]:
import pandas as pd
import numpy as np

In [2]:
df1=pd.read_csv("Customers.csv")
df2=pd.read_csv("Transactions.csv")
df=pd.merge(df1,df2,on='CustomerID',how='left')

In [3]:
df

,CustomerID,CustomerName,Region,SignupDate,TransactionID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,C0001,Lawrence Carroll,South America,2022-07-10,T00015,P054,2024-01-19 03:12:55,2.0,114.60,57.30
1,C0001,Lawrence Carroll,South America,2022-07-10,T00932,P022,2024-09-17 09:01:18,3.0,412.62,137.54
2,C0001,Lawrence Carroll,South America,2022-07-10,T00085,P096,2024-04-08 00:01:00,2.0,614.94,307.47
3,C0001,Lawrence Carroll,South America,2022-07-10,T00445,P083,2024-05-07 03:11:44,2.0,911.44,455.72
4,C0001,Lawrence Carroll,South America,2022-07-10,T00436,P029,2024-11-02 17:04:16,3.0,1300.92,433.64
...,...,...,...,...,...,...,...,...,...,...
996,C0200,Kelly Cross,Asia,2023-06-11,T00518,P034,2024-12-11 03:05:50,4.0,868.20,217.05
997,C0200,Kelly Cross,Asia,2023-06-11,T00091,P057,2024-04-27 19:06:20,1.0,239.70,239.70
998,C0200,Kelly Cross,Asia,2023-06-11,T00731,P061,2024-07-15 20:36:28,4.0,627.84,156.96
999,C0200,Kelly Cross,Asia,2023-06-11,T00771,P048,2024-09-10 09:50:48,4.0,1665.60,416.40


In [4]:
customer_data=df.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    avg_spend=('TotalValue', 'mean'),
    num_transactions=('TransactionID', 'count')
).reset_index()
final_data=pd.merge(customer_data,df1,on='CustomerID')

In [5]:
final_data

,CustomerID,total_spend,avg_spend,num_transactions,CustomerName,Region,SignupDate
0,C0001,3354.52,670.904000,5,Lawrence Carroll,South America,2022-07-10
1,C0002,1862.74,465.685000,4,Elizabeth Lutz,Asia,2022-02-13
2,C0003,2725.38,681.345000,4,Michael Rivera,South America,2024-03-07
3,C0004,5354.88,669.360000,8,Kathleen Rodriguez,South America,2022-10-09
4,C0005,2034.24,678.080000,3,Laura Weber,Asia,2022-08-15
...,...,...,...,...,...,...,...
195,C0196,4982.88,1245.720000,4,Laura Watts,Europe,2022-06-07
196,C0197,1928.65,642.883333,3,Christina Harvey,Europe,2023-03-21
197,C0198,931.83,465.915000,2,Rebecca Ray,Europe,2022-02-27
198,C0199,1979.28,494.820000,4,Andrea Jenkins,Europe,2022-12-03


In [6]:
final_data.isnull().sum()

CustomerID          0
total_spend         0
avg_spend           1
num_transactions    0
CustomerName        0
Region              0
SignupDate          0
dtype: int64

In [7]:
final_data.dropna()

,CustomerID,total_spend,avg_spend,num_transactions,CustomerName,Region,SignupDate
0,C0001,3354.52,670.904000,5,Lawrence Carroll,South America,2022-07-10
1,C0002,1862.74,465.685000,4,Elizabeth Lutz,Asia,2022-02-13
2,C0003,2725.38,681.345000,4,Michael Rivera,South America,2024-03-07
3,C0004,5354.88,669.360000,8,Kathleen Rodriguez,South America,2022-10-09
4,C0005,2034.24,678.080000,3,Laura Weber,Asia,2022-08-15
...,...,...,...,...,...,...,...
195,C0196,4982.88,1245.720000,4,Laura Watts,Europe,2022-06-07
196,C0197,1928.65,642.883333,3,Christina Harvey,Europe,2023-03-21
197,C0198,931.83,465.915000,2,Rebecca Ray,Europe,2022-02-27
198,C0199,1979.28,494.820000,4,Andrea Jenkins,Europe,2022-12-03


In [13]:
final_data=final_data.fillna(0)

In [14]:
final_data.isnull().sum()

CustomerID          0
total_spend         0
avg_spend           0
num_transactions    0
CustomerName        0
Region              0
SignupDate          0
dtype: int64

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
features = final_data[['total_spend', 'avg_spend', 'num_transactions']]
similarity_matrix = cosine_similarity(features)
similarity_df = pd.DataFrame(similarity_matrix, columns=final_data['CustomerID'], index=final_data['CustomerID'])

In [17]:
recommendations = {}

for customer in final_data['CustomerID']:
    similar_customers = similarity_df[customer].sort_values(ascending=False)[1:4]
    recommendations[customer] = list(zip(similar_customers.index, similar_customers.values))

recommendations_df = pd.DataFrame(recommendations.items(), columns=['CustomerID', 'Lookalikes'])
recommendations_df.to_csv('FirstName_LastName_Lookalike.csv', index=False)
